In [2]:
%%capture
!pip install peft
!pip install bitsandbytes
!pip install lightning

In [ ]:
import os
os._exit(00)

In [1]:
import json
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.optim import AdamW
import lightning as L
import gc
from lightning.pytorch.callbacks import ModelCheckpoint


/Users/samu/Desktop/Usi/Advanced Topics in Machine Learning/ATML_2/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

DATASET_PATH = "/kaggle/input/animedataset/anime-dataset-2023.csv"

anime_df = pd.read_csv(DATASET_PATH)

def prepare_alpaca_format_name_only(df):
    examples = []
    for _, row in df.iterrows():
        instruction = "Describe this anime"
        input_text = row['Name']
        output_text = row['Synopsis']
        
        example = {
            "instruction": instruction,
            "input": input_text,
            "output": output_text
        }
        examples.append(example)
    
    return examples

alpaca_dataset_name_only = prepare_alpaca_format_name_only(anime_df)

print(alpaca_dataset_name_only[0])

{'instruction': 'Describe this anime', 'input': 'Cowboy Bebop', 'output': "Crime is timeless. By the year 2071, humanity has expanded across the galaxy, filling the surface of other planets with settlements like those on Earth. These new societies are plagued by murder, drug use, and theft, and intergalactic outlaws are hunted by a growing number of tough bounty hunters.\n\nSpike Spiegel and Jet Black pursue criminals throughout space to make a humble living. Beneath his goofy and aloof demeanor, Spike is haunted by the weight of his violent past. Meanwhile, Jet manages his own troubled memories while taking care of Spike and the Bebop, their ship. The duo is joined by the beautiful con artist Faye Valentine, odd child Edward Wong Hau Pepelu Tivrusky IV, and Ein, a bioengineered Welsh Corgi.\n\nWhile developing bonds and working to catch a colorful cast of criminals, the Bebop crew's lives are disrupted by a menace from Spike's past. As a rival's maniacal plot continues to unravel, Spi

In [10]:
alpaca_data = alpaca_dataset_name_only

# Trasformiamo il dataset in un formato compatibile
train_formatted = [f"### Instruction:\n{sample['instruction']}\n\n### Input:\n{sample['input']}\n\n### Response:\n{sample['output']}" for sample in alpaca_data]

# Creiamo il dataset HuggingFace
train_dataset = Dataset.from_dict({"text": train_formatted})

# ####################################
# STEP 2.1: Configurazione della quantizzazione
# ####################################
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Carichiamo il modello in 4-bit
    bnb_4bit_compute_dtype=torch.bfloat16,  # Utilizziamo bfloat16 per i calcoli
    bnb_4bit_quant_type="nf4",              # Tipo di quantizzazione
    bnb_4bit_use_double_quant=True          # Abilitiamo la doppia quantizzazione
)

# ####################################
# STEP 2.2: Caricamento del Modello e Tokenizer
# ####################################
base_model_id = "PY007/TinyLlama-1.1B-Chat-v0.1"

# Carichiamo il tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token

# Carichiamo il modello con la configurazione quantizzata
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",          # Mappatura automatica sui dispositivi
    quantization_config=bnb_config
)

model.gradient_checkpointing_enable()

# ####################################
# STEP 3: Configurazione LoRa
# ####################################
lora_config = LoraConfig(
    r=8,                       # Grado di fattorizzazione per LoRa
    lora_alpha=16,             # Valore di alpha
    lora_dropout=0.05,         # Dropout per regolarizzare
    bias="none",              # Nessuna modifica ai bias
    task_type="CAUSAL_LM"      # Modello di linguaggio causale
)

# Prepariamo il modello per l'addestramento k-bit
model = prepare_model_for_kbit_training(model) 
model = get_peft_model(model, lora_config)

# ####################################
# STEP 4: DataLoader
# ####################################
def collate_fn(batch):
    inputs = tokenizer(
        [sample['text'] for sample in batch], 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=206
    )
    labels = inputs.input_ids.clone()
    labels[~inputs.attention_mask.bool()] = -100
    return inputs, labels

train_loader = DataLoader(
    train_dataset, 
    collate_fn=collate_fn, 
    shuffle=True, 
    batch_size=8, 
    num_workers=4
)

# ####################################
# STEP 4.1: Lightning Wrapper
# ####################################
class LightningWrapper(L.LightningModule):
    def __init__(self, model, lr=1e-4):
        super().__init__()
        self.model = model
        self.lr = lr

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.lr)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(**inputs)
        # Shift logits to exclude the last element
        # shift labels to exclude the first element
        logits = outputs.logits[..., :-1, :].contiguous()
        labels = labels[..., 1:].contiguous()
        # Compute LM loss token-wise
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1))
        self.log("train_loss", loss)
        return loss

lightning_model = LightningWrapper(model)

# ####################################
# STEP 4.2: Trainer + Train
# ####################################

# Callback per il salvataggio del checkpoint
checkpoint_callback = ModelCheckpoint(
    dirpath="./checkpoints",  
    filename="finetuned_model-{epoch:02d}-{train_loss:.2f}", 
    save_top_k=-1,            
    save_last=True,          
    monitor="train_loss",    
    mode="min"              
)

trainer = L.Trainer(
    accumulate_grad_batches=32,
    precision="bf16-mixed",
    gradient_clip_val=1.0,
    max_epochs=1,
    callbacks=[checkpoint_callback],  # Aggiungi il callback per i checkpoint
)

gc.collect()
torch.cuda.empty_cache()
trainer.fit(lightning_model, train_dataloaders=train_loader)


# ####################################
# STEP 6: Save the Fine-tuned Model
# ####################################
model.save_pretrained("./finetuned_qlora_model")
tokenizer.save_pretrained("./finetuned_qlora_model")



/opt/conda/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /kaggle/working/checkpoints exists and is not empty.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type                 | Params | Mode 
-------------------------------------------------------
0 | model | PeftModelForCausalLM | 616 M  | train
-------------------------------------------------------
1.1 M     Trainable params
615 M     Non-trainable params
616 M     Total params
2,466.947 Total estimated model params size (MB)
442       Modules in train mode
315       Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

('./finetuned_qlora_model/tokenizer_config.json',
 './finetuned_qlora_model/special_tokens_map.json',
 './finetuned_qlora_model/tokenizer.model',
 './finetuned_qlora_model/added_tokens.json',
 './finetuned_qlora_model/tokenizer.json')

In [ ]:
# Caricamento del checkpoint
checkpoint_path = "./checkpoints/last.ckpt"  # Percorso del checkpoint più recente

# Carica il modello Lightning dal checkpoint
lightning_model = LightningWrapper.load_from_checkpoint(checkpoint_path, model=model)

# Crea un nuovo Trainer
trainer = L.Trainer(
    accumulate_grad_batches=32,
    precision="bf16-mixed",
    gradient_clip_val=1.0,
    max_epochs=5  # Puoi aumentare il numero di epoch per continuare
)

trainer.fit(lightning_model, train_dataloaders=train_loader)


In [12]:
!zip -r file.zip /kaggle/working/checkpoints


  adding: kaggle/working/checkpoints/ (stored 0%)
  adding: kaggle/working/checkpoints/last.ckpt (deflated 22%)
  adding: kaggle/working/checkpoints/finetuned_model-epoch=00-train_loss=2.09.ckpt (deflated 22%)
  adding: kaggle/working/checkpoints/finetuned_model-epoch=00-train_loss=2.14.ckpt (deflated 22%)
  adding: kaggle/working/checkpoints/last-v1.ckpt (deflated 22%)


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("mps") 

model_path = "Models/Anime_Model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

def generate_response(instruction, query, model, tokenizer, max_new_tokens=100):
    prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{query}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens, 
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.9,
        top_p=0.9
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("### Response:")[-1].strip()

instruction = "Describe this anime"
query = "Neon Genesis Evangelion"

response = generate_response(instruction, query, model, tokenizer)
print("out:", response)


out: Anime of the franchise "Neon Genesis Evangelion". This is the first anime in the franchise, and the series is set during the events of the movie "Godzilla vs. Mecha Giga-Mars". The anime is about the struggle of a group of survivors who survived the destruction of the Earth after the fall of mankind.

(Source: Anime News Network)

Description:
An earth-shatter
